In [2]:
import pandas as pd

In [3]:
original_data = pd.read_parquet("escooter_copy.parquet")

### Data Cleaning

In [4]:
df_org_data = original_data.copy()

In [5]:
# Transformed dtypes of Dataframe   
df_org_data.holiday = df_org_data.holiday.astype(bool)
df_org_data.workingday = df_org_data.workingday.astype(bool)
df_org_data.loc[df_org_data.temp >45, "temp"] = df_org_data.temp.div(100)

In [6]:
# Dataframe as no NaN values
df_org_data.isnull().describe()

,datetime,holiday,workingday,weather,temp,atemp,humidity,windspeed,registered_customer
count,3755415,3755415,3755415,3755415,3755415,3755415,3755415,3755415,3755415
unique,1,1,1,1,1,1,1,1,1
top,False,False,False,False,False,False,False,False,False
freq,3755415,3755415,3755415,3755415,3755415,3755415,3755415,3755415,3755415


### Duplicates

In [7]:
df_org_data.count(axis=0)

datetime               3755415
holiday                3755415
workingday             3755415
weather                3755415
temp                   3755415
atemp                  3755415
humidity               3755415
windspeed              3755415
registered_customer    3755415
dtype: int64

In [8]:
df_org_data.nunique()

datetime               3565296
holiday                      2
workingday                   2
weather                      4
temp                        50
atemp                       65
humidity                    89
windspeed                   30
registered_customer          2
dtype: int64

In [9]:
# Duplicates ??
df_org_data.drop_duplicates().count(axis=0)

datetime               3610562
holiday                3610562
workingday             3610562
weather                3610562
temp                   3610562
atemp                  3610562
humidity               3610562
windspeed              3610562
registered_customer    3610562
dtype: int64

### Temp Fehler

In [10]:
df_temp = original_data.copy()

In [11]:
# Temperatures that are too high
df_temp.loc[df_temp.temp >45]

,datetime,holiday,workingday,weather,temp,atemp,humidity,windspeed,registered_customer
2138,2020-01-06 10:10:53,0.0,1.0,"clear, few clouds",738.0,8.335,43.0,16.9979,True
2139,2020-01-06 10:13:12,0.0,1.0,"clear, few clouds",738.0,8.335,43.0,16.9979,False
2140,2020-01-06 10:13:43,0.0,1.0,"clear, few clouds",738.0,8.335,43.0,16.9979,True
2141,2020-01-06 10:14:04,0.0,1.0,"clear, few clouds",738.0,8.335,43.0,16.9979,True
2142,2020-01-06 10:16:25,0.0,1.0,"clear, few clouds",738.0,8.335,43.0,16.9979,True
...,...,...,...,...,...,...,...,...,...
3751388,2022-01-03 10:18:44,0.0,1.0,"cloudy, mist",820.0,10.605,69.0,8.9981,True
3751389,2022-01-03 10:18:51,0.0,1.0,"cloudy, mist",820.0,10.605,69.0,8.9981,True
3751390,2022-01-03 10:18:59,0.0,1.0,"cloudy, mist",820.0,10.605,69.0,8.9981,False
3751391,2022-01-03 10:19:11,0.0,1.0,"cloudy, mist",820.0,10.605,69.0,8.9981,True


In [12]:
df_temp.loc[(df_temp.temp >45) & (df_temp.temp.isnull() == True)]

,datetime,holiday,workingday,weather,temp,atemp,humidity,windspeed,registered_customer


In [13]:
temp_mistake = df_temp.loc[df_temp.temp >45].groupby(pd.Grouper(key="datetime", freq="S")).agg({"weather": "count","humidity": "mean","windspeed": "mean","workingday": "sum","holiday": "sum", "temp": "mean", "atemp": "mean", "registered_customer": "count"}).dropna()

In [14]:
# 10:00:00 AM is the time
temp_mistake.sample(50)

,weather,humidity,windspeed,workingday,holiday,temp,atemp,registered_customer
datetime,,,,,,,,
2021-03-08 10:18:18,1,60.0,0.0000,1.0,0.0,984.0,12.880,1
2021-05-03 10:17:43,1,41.0,11.0014,1.0,0.0,1722.0,21.210,1
2021-04-05 10:18:26,1,50.0,35.0008,1.0,0.0,1640.0,20.455,1
2020-07-13 10:13:28,1,55.0,23.9994,1.0,0.0,3280.0,37.880,1
2021-12-13 10:12:07,1,94.0,15.0013,1.0,0.0,1804.0,21.970,1
2021-10-04 10:11:22,1,56.0,0.0000,1.0,0.0,2214.0,25.760,1
2020-10-19 10:13:37,1,45.0,11.0014,1.0,0.0,2132.0,25.000,1
2021-01-04 10:10:57,1,35.0,19.0012,0.0,1.0,1066.0,12.120,1
2021-04-26 10:17:17,1,70.0,22.0028,1.0,0.0,1312.0,15.150,1


### Humidity

In [15]:
dfh = df_org_data.copy()

In [16]:
# Max and Min of Humidity are extreme ?
dfh.humidity.describe().round(2)

count    3755415.00
mean          57.22
std           18.40
min            0.00
25%           43.00
50%           56.00
75%           71.00
max          100.00
Name: humidity, dtype: float64

In [17]:
# no NaN values
dfh.loc[dfh.humidity.isna() ==  True]


,datetime,holiday,workingday,weather,temp,atemp,humidity,windspeed,registered_customer


In [18]:
dfh1 = dfh.loc[dfh.humidity == 100]
dfh1[["datetime", "temp", "atemp", "weather", "windspeed"]].to_clipboard()

In [19]:
dfh2 = dfh1.groupby(["weather", "datetime"]).agg(temp =("temp", "mean"), atemp= ("atemp", "mean"), windspeed= ("windspeed", "mean"), humidity= ("humidity", "mean")).reset_index()
dfh2

,weather,datetime,temp,atemp,windspeed,humidity
0,"clear, few clouds",2020-02-08 21:00:28,10.66,12.88,12.998,100.0
1,"clear, few clouds",2020-02-08 21:01:47,10.66,12.88,12.998,100.0
2,"clear, few clouds",2020-02-08 21:02:14,10.66,12.88,12.998,100.0
3,"clear, few clouds",2020-02-08 21:03:10,10.66,12.88,12.998,100.0
4,"clear, few clouds",2020-02-08 21:03:24,10.66,12.88,12.998,100.0
...,...,...,...,...,...,...
19528,light snow or rain or thunderstorm,2022-01-01 13:59:25,8.20,12.12,0.000,100.0
19529,light snow or rain or thunderstorm,2022-01-01 13:59:28,8.20,12.12,0.000,100.0
19530,light snow or rain or thunderstorm,2022-01-01 13:59:59,8.20,12.12,0.000,100.0
19531,light snow or rain or thunderstorm,2022-01-01 14:00:17,8.20,12.12,0.000,100.0


In [20]:
dfh3 = dfh2.loc[dfh2.weather == "clear, few clouds"].reset_index(drop = True)

In [21]:
dfh3.loc[dfh3.datetime.between("2020-12-1", "2020-12-31")].head(5)

,weather,datetime,temp,atemp,windspeed,humidity
807,"clear, few clouds",2020-12-09 00:02:06,18.04,21.97,12.998,100.0
808,"clear, few clouds",2020-12-09 00:02:55,18.04,21.97,12.998,100.0
809,"clear, few clouds",2020-12-09 00:04:53,18.04,21.97,12.998,100.0
810,"clear, few clouds",2020-12-09 00:05:07,18.04,21.97,12.998,100.0
811,"clear, few clouds",2020-12-09 00:06:10,18.04,21.97,12.998,100.0


In [22]:
dfh3.to_clipboard()

woher bezeiehen sie die wetterdaten (großraumwetterdaten)?
wie genau ist der humidity sensor, welche toleranz hat er ?
gibt es eine bestimmte geographische region der die daten entstammen? 

clear, few clouds	2020-02-08 21:54:54	10.66	12.88	12.998	100.0 kann zb nicht schwül sein da die temp dafür zu niedrig ist! 

In [23]:
dfh4 = dfh2.loc[dfh2.humidity == 0]

In [24]:
dfh_0 = dfh.loc[dfh.humidity == 0]

In [38]:
dfh01 = dfh_0.groupby(["weather", "datetime"]).agg(temp =("temp", "mean"), atemp= ("atemp", "mean"), windspeed= ("windspeed", "mean"), humidity= ("humidity", "mean"))
dfh01

temp   atemp  \
weather                            datetime                             
cloudy, mist                       2020-03-12 17:01:00  18.04  21.970   
                                   2020-03-12 17:02:38  18.04  21.970   
                                   2020-03-12 17:02:52  18.04  21.970   
                                   2020-03-12 17:03:00  18.04  21.970   
                                   2020-03-12 17:03:55  18.04  21.970   
...                                                       ...     ...   
light snow or rain or thunderstorm 2020-03-12 23:54:20  13.94  17.425   
                                   2020-03-12 23:54:25  13.94  17.425   
                                   2020-03-12 23:55:02  13.94  17.425   
                                   2020-03-12 23:55:12  13.94  17.425   
                                   2020-03-12 23:59:02  13.94  17.425   

                                                        windspeed  humidity  
weather                            datetime                                  
cloudy, mist                       2020-03-12 17:01:00    26.0027       0.0  
                                   2020-03-12 17:02:38    26.0027       0.0  
                                   2020-03-12 17:02:52    26.0027       0.0  
                                   2020-03-12 17:03:00    26.0027       0.0  
                                   2020-03-12 17:03:55    26.0027       0.0  
...                                                           ...       ...  
light snow or rain or thunderstorm 2020-03-12 23:54:20     6.0032       0.0  
                                   2020-03-12 23:54:25     6.0032       0.0  
                                   2020-03-12 23:55:02     6.0032       0.0  
                                   2020-03-12 23:55:12     6.0032       0.0  
                                   2020-03-12 23:59:02     6.0032       0.0  

[620 rows x 4 columns]

In [39]:
dfh02 = dfh_0.groupby("weather").agg(temp =("temp", "mean"), atemp= ("atemp", "mean"), windspeed= ("windspeed", "mean"), humidity= ("humidity", "mean"))
dfh02

,temp,atemp,windspeed,humidity
weather,,,,
"cloudy, mist",17.078621,20.726069,22.015414,0.0
light snow or rain or thunderstorm,16.490921,20.130680,20.078232,0.0
